In [1]:
import numpy as np
import pandas as pd
import keras
import tensorflow as tf
from sklearn import metrics
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input,LeakyReLU, Flatten, MaxPooling1D, Conv1D, MaxPool1D, GlobalMaxPool1D, Dense, AveragePooling1D, GlobalAveragePooling1D

In [2]:
import matplotlib
matplotlib.use('nbagg')
import matplotlib.pyplot as plt
%matplotlib tk

In [3]:
import os, os.path, sys

In [4]:
path = r"C:\Users\mata2\Desktop\master\podaci\0k\Y"

In [5]:
dir = os.listdir(path)

Ucitavanje fajlova

In [6]:
file_names = []
for file in dir:
    file_names.append(file)

In [7]:
percent_train = 0.8

In [8]:
percent_val = 0.2

Konkatenacija podataka koji se posle koriste za adekvatnu raspodelu na skup za treniranje i validaciju

In [9]:
train_files_names = file_names[:int(percent_train*len(file_names))]
#test_file_names = file_names[int(percent_train*len(file_names))::]

Ucitavanje zadate trajektorije

In [10]:
y_command = np.asarray(pd.read_csv(r"C:\Users\mata2\Desktop\master\podaci\yCmd.txt",header=None).astype('int'))

In [11]:
train_ = np.concatenate([pd.read_csv(path + "/" + file, header=None)[:61000].astype('int')
                              for file in train_files_names], axis=0)
# test_ = np.concatenate([pd.read_csv(path + "/" + file, header=None)[:61000].astype('int')
#                               for file in test_file_names], axis=0)

Ucitavanje stadardnih devijacija i strednih vrednosti

In [12]:
std_dev = []
with open(r"C:\Users\mata2\Desktop\master\Axis-projection-RNN\odstupanja\0k Y\Y_std_dev.txt", 'r') as lines:
    for line in lines:
        std_dev.append(float(line.strip('\n')))

In [13]:
sr_vrednost = []
with open(r"C:\Users\mata2\Desktop\master\Axis-projection-RNN\odstupanja\0k Y\Y_sr_vrednost.txt", 'r') as lines:
    for line in lines:
        sr_vrednost.append(float(line.strip('\n')))

Raspodela podataka kako bi se izbegla fiktivna stanja

In [14]:
for j in range(len(train_files_names)):
    exec(f"razlika_trening_{j} = []")
    for i in range(len(y_command)):
        exec(f"razlika_trening_{j}.append(y_command[i] - train_[j*len(y_command)+i])")
    exec(f'razlika_trening_{j} = np.asarray(razlika_trening_{j})')        

In [16]:
def create_timeseries_split(data, time_steps):

    # Get the number of samples in the data
    num_samples = len(data) - time_steps

    # Create empty arrays to store features and target values
    x_train = np.zeros((num_samples, time_steps, 1))
    y_train = np.zeros((num_samples, 1))

    # Loop through the data and create features and target values
    for i in range(num_samples):
        # Extract a slice of data for the current feature
        x_train[i] = data[i:i+time_steps, :]

        # The target value is the next value after the feature
        y_train[i] = data[i+time_steps, 0]

    return x_train, y_train

# TESTIRANJE ARHITEKTURE i LR/BATCH

radjeno u vise iteracija sa razlicitim vrednostima zbog vremenske kompleksnosti

In [17]:
filter_size = [16,32]
kernel_size_1 = [3,4]
pooling_size = [3,4]
out_size = [8,16]
time_steps_list = [8,10]

In [18]:
best_skor = float('-inf')
best_filter = None
best_kernel_1 = None
best_pool = None
best_out = None
best_step = None
loop = 0

for step in time_steps_list:
    
    #delimo podatke na osnovu velicine buffera
    input_shape = (step,1)

    for i in range(len(train_files_names)):
        exec(f'train_X_{i}, train_y_{i} = create_timeseries_split(razlika_trening_{i}, step)')

    train_X = np.asarray(train_X_0)
    for i in range(1,len(train_files_names)):
        exec(f'train_X = np.append(train_X, train_X_{i}, axis=0)')

    train_y = np.asarray(train_y_0)
    for i in range(1,len(train_files_names)):
        exec(f'train_y = np.append(train_y, train_y_{i}, axis=0)')  
    
    #uzimamo percent_val sa kraja train_X skupa 
    val_x = train_X[-int(len(train_X)*percent_val)-1:-1]
    val_y = train_y[-int(len(train_X)*percent_val)-1:-1]

    train_X = train_X[:len(train_X)-val_x.shape[0]]
    train_y = train_y[:len(train_y)-val_x.shape[0]]
         
    for filt in filter_size:
        for kernel_1 in kernel_size_1:
            for pool in pooling_size:
                for out in out_size:
                    keras.utils.set_random_seed(7)
                    optimizerMSE = keras.optimizers.Adam(learning_rate=0.0001)

                    modelMSE = Sequential()
                    modelMSE.add(Input(shape=input_shape))
                    modelMSE.add(Conv1D(filters=filt, kernel_size=kernel_1,padding='same', activation=LeakyReLU()))
                    modelMSE.add(MaxPooling1D(pool_size=pool))
                    modelMSE.add(Flatten())
                    modelMSE.add(Dense(out, activation=LeakyReLU()))
                    modelMSE.add(Dense(units = 1, activation=LeakyReLU()))
                    modelMSE.compile(loss='mean_squared_error', optimizer = optimizerMSE)

                    modelMSE.fit(train_X, train_y, epochs = 10, batch_size = 256, verbose = 2, shuffle = False)
                    test_predict_MSE = modelMSE.predict(val_x)

                    #zato sto smo uzeli 20% od kraja, ne pocinje indeks od samog pocetka rada masine
                    #radimo evaluaciju na mean 3 sigma testu
                    i =(len(y_command)-step) - val_x.shape[0]%(len(y_command)-step) -1
                    mean_3std = 0
                    for j in range(len(test_predict_MSE)):
                        if i%len(y_command) == 0:
                            i = step
                        if sr_vrednost[i] - 3*std_dev[i] <= test_predict_MSE[j] <= sr_vrednost[i] + 3*std_dev[i]:
                                mean_3std += 1
                        i += 1



                    if mean_3std > best_skor:
                        best_skor = mean_3std
                        best_filter = filt
                        best_kernel_1 = kernel_1
                        best_pool = pool
                        best_step = step
                        best_out = out

                        print('best_skor: ', best_skor)
                        print('best_filter: ', best_filter)
                        print('best_kernel_1: ', best_kernel_1)
                        print('best_step: ', best_step)
                        print('best_out: ', best_out)
                        print('best_pool', best_pool)
                        
                    loop += 1
                    print("loop: ", loop)
                                    
print('best_skor: ', best_skor)
print('best_filter: ', best_filter)
print('best_kernel_1: ', best_kernel_1)
print('best_step', best_step)
print('best_out: ', best_out)
print('best_pool', best_pool)



Epoch 1/10

4575/4575 - 9s - loss: 27.1109 - 9s/epoch - 2ms/step
Epoch 2/10
4575/4575 - 7s - loss: 10.6351 - 7s/epoch - 2ms/step
Epoch 3/10
4575/4575 - 7s - loss: 8.0074 - 7s/epoch - 2ms/step
Epoch 4/10
4575/4575 - 7s - loss: 7.2862 - 7s/epoch - 2ms/step
Epoch 5/10
4575/4575 - 7s - loss: 6.8429 - 7s/epoch - 2ms/step
Epoch 6/10
4575/4575 - 7s - loss: 6.5244 - 7s/epoch - 2ms/step
Epoch 7/10
4575/4575 - 7s - loss: 6.2947 - 7s/epoch - 2ms/step
Epoch 8/10
4575/4575 - 7s - loss: 6.1254 - 7s/epoch - 2ms/step
Epoch 9/10
4575/4575 - 7s - loss: 5.9895 - 7s/epoch - 2ms/step
Epoch 10/10
4575/4575 - 7s - loss: 5.8724 - 7s/epoch - 2ms/step
9149/9149 [==============================] - 9s 991us/step
best_skor:  280297
best_filter:  16
best_kernel_1:  3
best_step:  8
best_out:  8
best_pool 3
loop:  1
Epoch 1/10
4575/4575 - 8s - loss: 22.9261 - 8s/epoch - 2ms/step
Epoch 2/10
4575/4575 - 7s - loss: 9.2945 - 7s/epoch - 2ms/step
Epoch 3/10
4575/4575 - 7s - loss: 7.4700 - 7s/epoch - 2ms/step
Epoch 4/10
45

Epoch 8/10
4575/4575 - 8s - loss: 2.8545 - 8s/epoch - 2ms/step
Epoch 9/10
4575/4575 - 8s - loss: 2.7732 - 8s/epoch - 2ms/step
Epoch 10/10
4575/4575 - 8s - loss: 2.7078 - 8s/epoch - 2ms/step
9149/9149 [==============================] - 9s 994us/step
loop:  11
Epoch 1/10
4575/4575 - 9s - loss: 17.4891 - 9s/epoch - 2ms/step
Epoch 2/10
4575/4575 - 8s - loss: 5.0797 - 8s/epoch - 2ms/step
Epoch 3/10
4575/4575 - 8s - loss: 3.7884 - 8s/epoch - 2ms/step
Epoch 4/10
4575/4575 - 8s - loss: 3.3232 - 8s/epoch - 2ms/step
Epoch 5/10
4575/4575 - 8s - loss: 3.0462 - 8s/epoch - 2ms/step
Epoch 6/10
4575/4575 - 8s - loss: 2.8551 - 8s/epoch - 2ms/step
Epoch 7/10
4575/4575 - 8s - loss: 2.7065 - 8s/epoch - 2ms/step
Epoch 8/10
4575/4575 - 8s - loss: 2.5871 - 8s/epoch - 2ms/step
Epoch 9/10
4575/4575 - 8s - loss: 2.4977 - 8s/epoch - 2ms/step
Epoch 10/10
4575/4575 - 8s - loss: 2.4286 - 8s/epoch - 2ms/step
9149/9149 [==============================] - 9s 997us/step
loop:  12
Epoch 1/10
4575/4575 - 9s - loss: 17.253

Epoch 2/10
4575/4575 - 8s - loss: 5.9004 - 8s/epoch - 2ms/step
Epoch 3/10
4575/4575 - 8s - loss: 4.7159 - 8s/epoch - 2ms/step
Epoch 4/10
4575/4575 - 8s - loss: 4.0407 - 8s/epoch - 2ms/step
Epoch 5/10
4575/4575 - 8s - loss: 3.5823 - 8s/epoch - 2ms/step
Epoch 6/10
4575/4575 - 8s - loss: 3.2778 - 8s/epoch - 2ms/step
Epoch 7/10
4575/4575 - 8s - loss: 3.0756 - 8s/epoch - 2ms/step
Epoch 8/10
4575/4575 - 8s - loss: 2.9251 - 8s/epoch - 2ms/step
Epoch 9/10
4575/4575 - 8s - loss: 2.8031 - 8s/epoch - 2ms/step
Epoch 10/10
4575/4575 - 8s - loss: 2.6967 - 8s/epoch - 2ms/step
9149/9149 [==============================] - 9s 983us/step
loop:  23
Epoch 1/10
4575/4575 - 9s - loss: 14.4114 - 9s/epoch - 2ms/step
Epoch 2/10
4575/4575 - 8s - loss: 5.3805 - 8s/epoch - 2ms/step
Epoch 3/10
4575/4575 - 8s - loss: 4.2871 - 8s/epoch - 2ms/step
Epoch 4/10
4575/4575 - 8s - loss: 3.6784 - 8s/epoch - 2ms/step
Epoch 5/10
4575/4575 - 8s - loss: 3.2381 - 8s/epoch - 2ms/step
Epoch 6/10
4575/4575 - 8s - loss: 2.9884 - 8s/e

In [19]:
time_steps = 10
input_shape = (time_steps,1)

In [20]:
for i in range(len(train_files_names)):
    exec(f'train_X_{i}, train_y_{i} = create_timeseries_split(razlika_trening_{i}, time_steps)')
    
train_X = np.asarray(train_X_0)
for i in range(1,len(train_files_names)):
    exec(f'train_X = np.append(train_X, train_X_{i}, axis=0)')

train_y = np.asarray(train_y_0)
for i in range(1,len(train_files_names)):
    exec(f'train_y = np.append(train_y, train_y_{i}, axis=0)') 

In [22]:
val_x = train_X[-int(len(train_X)*percent_val)-1:-1]
val_y = train_y[-int(len(train_X)*percent_val)-1:-1]

In [23]:
train_X = train_X[:len(train_X)-val_x.shape[0]]
train_y = train_y[:len(train_y)-val_x.shape[0]]

In [24]:
keras.utils.set_random_seed(7)
optimizerMSE = keras.optimizers.Adam(learning_rate=0.0001)

modelMSE = Sequential()
modelMSE.add(Input(shape=(time_steps, 1)))
modelMSE.add(Conv1D(filters=16, kernel_size=3,padding='same', activation=LeakyReLU()))
modelMSE.add(MaxPooling1D(pool_size=4))
modelMSE.add(Flatten())
modelMSE.add(Dense(8, activation=LeakyReLU()))
modelMSE.add(Dense(units = 1, activation=LeakyReLU()))
modelMSE.compile(loss='mean_squared_error', optimizer = optimizerMSE)

In [25]:
historyMSE = modelMSE.fit(train_X, train_y, epochs=10, batch_size = 256, shuffle=False, verbose = 2)

Epoch 1/10
4575/4575 - 9s - loss: 22.5639 - 9s/epoch - 2ms/step
Epoch 2/10
4575/4575 - 8s - loss: 9.7526 - 8s/epoch - 2ms/step
Epoch 3/10
4575/4575 - 8s - loss: 7.9689 - 8s/epoch - 2ms/step
Epoch 4/10
4575/4575 - 8s - loss: 7.2953 - 8s/epoch - 2ms/step
Epoch 5/10
4575/4575 - 8s - loss: 6.8698 - 8s/epoch - 2ms/step
Epoch 6/10
4575/4575 - 8s - loss: 6.5223 - 8s/epoch - 2ms/step
Epoch 7/10
4575/4575 - 8s - loss: 6.2267 - 8s/epoch - 2ms/step
Epoch 8/10
4575/4575 - 8s - loss: 5.9589 - 8s/epoch - 2ms/step
Epoch 9/10
4575/4575 - 8s - loss: 5.6736 - 8s/epoch - 2ms/step
Epoch 10/10
4575/4575 - 8s - loss: 5.4338 - 8s/epoch - 2ms/step


In [27]:
modelMSE.summary()

Model: "sequential_32"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_32 (Conv1D)          (None, 10, 16)            64        
                                                                 
 max_pooling1d_32 (MaxPooli  (None, 2, 16)             0         
 ng1D)                                                           
                                                                 
 flatten_32 (Flatten)        (None, 32)                0         
                                                                 
 dense_64 (Dense)            (None, 8)                 264       
                                                                 
 dense_65 (Dense)            (None, 1)                 9         
                                                                 
Total params: 337 (1.32 KB)
Trainable params: 337 (1.32 KB)
Non-trainable params: 0 (0.00 Byte)
_______________________

In [26]:
test_predict_MSE = modelMSE.predict(val_x)

9149/9149 [==============================] - 9s 1ms/step


In [29]:
i =(len(y_command)-time_steps) - val_x.shape[0]%(len(y_command)-time_steps) -1
mean_3std = 0
for j in range(len(test_predict_MSE)):
    if i%len(y_command) == 0:
        i = time_steps
    if sr_vrednost[i] - 3*std_dev[i] <= test_predict_MSE[j] <= sr_vrednost[i] + 3*std_dev[i]:
            mean_3std += 1
    i += 1
mean_3std/len(test_predict_MSE)

0.9843451112204187